# MNIST and CNN

## MNIST

### Load Dataset

In [1]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
train_data = dsets.MNIST(root='data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

test_data = dsets.MNIST(root='data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

### Make Batches

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [5]:
batch_size = 100

train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)

## Train Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, 5), # 1x28x28 -> 16x24x24
            nn.ReLU(),
            nn.Conv2d(16, 32, 5), # 16x24x24 -> 32x20x20
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 32x20x20 -> 
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )       
        
    def forward(self,x):
        out = self.conv_layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out

In [7]:
model = CNN().cuda()

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [9]:
num_epochs = 3

In [10]:
for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/3], lter [300/600], Loss: 2.2729
Epoch [1/3], lter [600/600], Loss: 0.9598
Epoch [2/3], lter [300/600], Loss: 0.3264
Epoch [2/3], lter [600/600], Loss: 0.2135
Epoch [3/3], lter [300/600], Loss: 0.3169
Epoch [3/3], lter [600/600], Loss: 0.1264


## Machine Unlearning

### Make Forget and Retain data

In [11]:
import numpy as np
from torch.utils.data import Subset

In [12]:
targets = train_data.targets.numpy()
forget_indices = np.where(targets == 1)[0]
retain_indices = np.where(targets != 1)[0]

In [13]:
forget_data = Subset(train_data, forget_indices)
retain_data = Subset(train_data, retain_indices)

In [14]:
forget_loader = DataLoader(dataset=forget_data, 
                           batch_size=batch_size,
                           shuffle=True)

retain_loader = DataLoader(dataset=retain_data, 
                           batch_size=batch_size,
                           shuffle=True)

In [15]:
@torch.no_grad()
def evaluate(model, loader):
    model = model.eval()
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.cuda()
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

In [16]:
print("- Forget:")
evaluate(model, forget_loader)
print("- Retain:")
evaluate(model, retain_loader)

- Forget:
Accuracy of test images: 97.641649 %
- Retain:
Accuracy of test images: 94.693755 %


### Unlearning (Finetuning)

In [17]:
from copy import deepcopy

In [18]:
model_ft = deepcopy(model).train()
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.01)

In [19]:
num_epochs = 3

In [20]:
for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size

    for i, (batch_images, batch_labels) in enumerate(retain_loader):  # Use retain loader

        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model_ft(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/3], lter [300/600], Loss: 0.2435
Epoch [2/3], lter [300/600], Loss: 0.0580
Epoch [3/3], lter [300/600], Loss: 0.0552


In [21]:
print("- Forget:")
evaluate(model_ft, forget_loader)
print("- Retain:")
evaluate(model_ft, retain_loader)

- Forget:
Accuracy of test images: 47.878968 %
- Retain:
Accuracy of test images: 97.283037 %


### Unlearning (Gradient Ascent)

In [72]:
from copy import deepcopy

In [73]:
model_ga = deepcopy(model).train()
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ga.parameters(), lr=0.00004)

In [74]:
num_epochs = 2

In [75]:
for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size

    for i, (batch_images, batch_labels) in enumerate(forget_loader):  # Use forget loader

        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model_ga(X)
        cost = -loss(pre, Y)  # Use negative loss

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

In [76]:
print("- Forget:")
evaluate(model_ga, forget_loader)
print("- Retain:")
evaluate(model_ga, retain_loader)

- Forget:
Accuracy of test images: 0.000000 %
- Retain:
Accuracy of test images: 71.989185 %
